In [3]:
import duckdb

con = duckdb.connect("../data/warehouse/olist.duckdb")


In [4]:
con.execute("SHOW TABLES").fetchall()


[('customers',),
 ('order_items',),
 ('orders',),
 ('payments',),
 ('products',),
 ('reviews',),
 ('sellers',)]

In [5]:
con.execute("DESCRIBE orders").fetchdf()


,column_name,column_type,null,key,default,extra
0,order_id,VARCHAR,YES,None,None,None
1,customer_id,VARCHAR,YES,None,None,None
2,order_status,VARCHAR,YES,None,None,None
3,order_purchase_timestamp,VARCHAR,YES,None,None,None
4,order_approved_at,VARCHAR,YES,None,None,None
5,order_delivered_carrier_date,VARCHAR,YES,None,None,None
6,order_delivered_customer_date,VARCHAR,YES,None,None,None
7,order_estimated_delivery_date,VARCHAR,YES,None,None,None
